# **LoopsResultsAnalysis**

## **1. Importing and First Proccesing**

In [1]:
import pandas as pd
from scipy import stats
import plotly.express as px
import ProcessingConfig as config
import ProcessingFuncs as process
import AnalysisFuncs as analyze

pd.options.display.max_columns = 50

In [2]:
raw_data = pd.read_excel(config.cleaning_config['raw_data_path'])

first_lines_data = process.clean_data(raw_data
                                      , drop_assign_steps=True
                                      , drop_first_loop_steps=True
                                      , only_first_lines=True
                                      , filter_subjects=True
                                      , filter_trials=True
                                      , filter_steps=True)
process.save_in_excel(first_lines_data)

analyze.get_sample_size(first_lines_data)

original shape: (20617, 32)
threshold for outliers detection: 3 (iqr).
drop_assign: 540 rows were filtered out.
drop_first_loop: 2532 rows were filtered out.
filter_slow_subjects: No slow subjects detected.
filter_bad_subjects: No bad subjects detected (in terms of low success rate).
is_first_line: 13764 lines were filtered.
filter_bad_trials: No bad trials detected (in terms of low success rate).
filter_slow_steps: 56 slow steps were filtered out.
Here is a summary of slow steps rate per subjects: 
     subject              101A  112A  111B  111A  110A  104A  102A  106A  107A  \
slow steps rate (%)   0.0   0.0   0.0   0.0   0.0  0.64  0.65  0.65  0.71   

subject              109B  102B  106B  110B  107B  103A  105A  104B  101B  \
slow steps rate (%)  1.29  1.29  1.31  1.32  1.33  1.35  2.05  2.19  2.63   

subject              105B  109A  108A  108B  103B  113A  112B  
slow steps rate (%)  2.65  2.74  3.25  3.42  3.55  3.57  4.76  
final shape: (3725, 26)
There are 45 sessions from 2

## **2. General Priming Effect Testing**

In [3]:
# mean response time by loop switching only
first_lines_data[first_lines_data['correct']].groupby(['loop_type_switch'])['rt'].mean().to_frame()

,rt
loop_type_switch,
False,3469.191382
True,3475.793704


In [4]:
# mean response time grouped by loop type and loop switching.
switching_diff_basic = pd.DataFrame()

switching_diff_basic['mean_response_time'] = first_lines_data[first_lines_data['correct']].groupby(['loop_type', 'loop_type_switch'])['rt'].mean()
switching_diff_basic['steps_counter'] = first_lines_data[first_lines_data['correct']].groupby(['loop_type', 'loop_type_switch'])['step_num'].count()

switching_diff_basic

mean_response_time  steps_counter
loop_type loop_type_switch                                   
for       False                    3328.269019            907
          True                     3304.788354            893
while     False                    3614.437500            880
          True                     3648.150113            886

In [27]:
switch_basic_for_fig = switching_diff_basic.reset_index()
px.bar(switch_basic_for_fig, x='loop_type', y='mean_response_time', color='loop_type_switch', barmode='group'
        , title='Priming Effect Per Loop Type', labels={'loop_type': 'after switch loop type'
                                                                      , 'mean_response_time': 'mean response time (ms)'
                                                                      , 'loop_type_switch': 'switching loop type'
                                                                      })

In [5]:
# checking for a significant pearson correlation between switching and response time
analyze.test_rt_switch_corr(first_lines_data)

Pearson correlation between loop type switching and response time 
          is not significant (p = 0.43), with value of r = 0.013


## **3. Different Influencors on Priming Effect**

#### **3.1. Mistake in Previous Step**

In [8]:
analyze.is_prev_correct(raw_data, first_lines_data)

There are 57 steps whose previous step is incorrect.


In [9]:
# mean response time grouped by loop switching and previous step correctness.
switching_diff_prev_correct = pd.DataFrame()

switching_diff_prev_correct['mean_response_time'] = first_lines_data[first_lines_data['correct']].groupby(['loop_type_switch', 'is_prev_correct'])['rt'].mean()
switching_diff_prev_correct['number_of_steps'] = first_lines_data[first_lines_data['correct']].groupby(['loop_type_switch', 'is_prev_correct'])['step_num'].count()

switching_diff_prev_correct

mean_response_time  number_of_steps
loop_type_switch is_prev_correct                                     
False            False                   4104.160000               25
                 True                    3460.182179             1762
True             False                   5263.363636               22
                 True                    3453.410928             1757

In [23]:
switch_prev_for_fig = switching_diff_prev_correct.reset_index()
px.bar(switch_prev_for_fig, x='is_prev_correct', y='mean_response_time', color='loop_type_switch', barmode='group'
        , title='Previous Step Mistake Influence On Priming', labels={'is_prev_correct': 'previous step correctness'
                                                                      , 'mean_response_time': 'mean response time (ms)'
                                                                      , 'loop_type_switch': 'switching loop type'
                                                                      })

## **4. Different Influencors on Response Time Generally**

In [10]:
all_lines_data = process.clean_data(raw_data
                                      , only_first_lines=False
                                      , filter_subjects=True
                                      , filter_trials=True
                                      , filter_steps=True)

original shape: (20617, 25)
threshold for outliers detection: 3 (iqr).
drop_assign: 540 rows were filtered out.
drop_first_loop: 2532 rows were filtered out.
filter_slow_subjects: No slow subjects detected.
filter_bad_subjects: No bad subjects detected (in terms of low success rate).
is_first_line: There are 3781 first lines over all.
filter_bad_trials: No bad trials detected (in terms of low success rate).
filter_slow_steps: 214 slow steps were filtered out.
Here is a summary of slow steps rate per subjects: 
     subject              104A  110A  113A  110B  107A  111B  111A  101A  102B  \
slow steps rate (%)  0.23  0.26  0.51  0.67  0.71  0.77  1.13  1.18  1.39   

subject              107B  106B  112A  102A  112B  105A  106A  104B  109A  \
slow steps rate (%)  1.41  1.42  1.55  1.69   1.8  1.88  2.27  2.35  2.53   

subject              108A  109B  105B  103A  101B  108B  103B  
slow steps rate (%)  2.55  2.68  2.84  3.02  4.26  5.32  5.78  
final shape: (17331, 26)


#### **4.2. Step Type - Arithmetics vs Loop Ends**

In [11]:
all_lines_data['is_loop_end'] = all_lines_data['loop_step'] == all_lines_data['n_iterations']

In [12]:
all_lines_data.groupby('is_loop_end').agg({'correct': 'mean', 'rt': 'mean', 'step_num': 'count'})

,correct,rt,step_num
is_loop_end,,,
False,0.963394,1865.844618,15626
True,0.964809,2022.941349,1705


#### **4.2. Mistake in Previous Step**

In [13]:
non_first_steps = all_lines_data[all_lines_data['step_num'] != 0] # filtering steps with no previous step
analyze.is_prev_correct(raw_data, non_first_steps)

There are 557 steps whose previous step is incorrect.


In [14]:
# checking for a difference in response time after a mistake
non_first_steps[non_first_steps['correct']].groupby('is_prev_correct')['rt'].agg(['mean', 'std', 'count'])

,mean,std,count
is_prev_correct,,,
False,1557.189300,1797.813420,486
True,1853.433541,1694.775063,16213
